This script animates results of the simulation as spread over time.

This is adapted from Gytis Dudas' (@evogytis) notebook curonia, part of baltic, which can be found here: https://github.com/evogytis/baltic

GeoJson data from Global Administrative Database

In [2]:
import matplotlib as mpl ## matplotlib should not be set to inline mode to accelerate animation rendering and save memory
mpl.use('Agg') ## recommended backend for animations
from matplotlib import pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import gridspec
import matplotlib.patheffects as path_effects
import matplotlib.animation as animation
from IPython.display import clear_output
from IPython.display import HTML

import requests
import json
from io import StringIO as sio

import numpy as np
import pandas as pd
from scipy.special import binom
import tkinter
mpl.use('TkAgg')

import datetime as dt
import time

from scipy.interpolate import UnivariateSpline ## used to smooth counts of lineages in each location at any given time
from scipy.interpolate import interp1d ## used to linearly interpolate between data points used in colouring polygons
from sklearn.decomposition import IncrementalPCA ## used to identify PCA1 when automatically producing a colour map

In [3]:
typeface='Helvetica Neue' ## set default matplotlib font and font size
mpl.rcParams['font.weight']=300
mpl.rcParams['axes.labelweight']=300
mpl.rcParams['font.family']=typeface
mpl.rcParams['font.size']=22

In [4]:
import sys

sys.path.insert(0,"../../Simulation_scripts/no_interventions")

import Tree_simulator as cts
from collections import defaultdict
##(trans_dict, nodes, sampling_proportion, epidemic_len)

dropbox_path = "/Users/s1743989/VirusEvolution Dropbox/Verity Hill/Agent_based_model/Looping models/"
results = "Results/Fitted_runs/no_caps/2/log_files/"
log_file = "information_file_for_40.csv"

### Prepping maps and locations

In [13]:
locations = ["bo", "bombali", "bonthe", 'kailahun', "kambia", 'kenema', 'koinadugu', 'kono', 'moyamba', 'portloko', 'pujehun', 'tonkolili', 'westernarearural', 'westernareaurban']

with open("shape_files/gadm36_SLE_2.json") as json_file:
    json_map = json.load(json_file)

In [28]:
features=json_map['features']
location_points={} ## location points will be stored here
polygons={} ## polygons will be stored here

loc_name='NAME_2' ## key name for each feature


for loc in features: ## iterate through features (locations)
    
    poly = np.asarray(loc['geometry']['coordinates']) ## get coordinates
    #location=removeDiacritics(loc['properties'][loc_name]) ## standardised location name (remove diacritics)
    location = (loc['properties'][loc_name]).lower()
    if location == "western rural":
        location = "westernarearural"
    elif location == "western urban":
        location = "westernareaurban"
    elif location == "port loko":
        location = "portloko"
#     print(location.encode().decode('utf-8'))

    polygons[location]=[]
    location_points[location]=[]
    if loc['geometry']['type']=='MultiPolygon': ## multiple parts detected
        for part in np.asarray(poly): ## iterate over each component polygon
            for coords in np.asarray(part): ## iterate over coordinates
                coords=np.array(coords)
                xs=coords[:,0] ## longitudes
                ys=coords[:,1] ## latitudes

                location_points[location].append(np.vstack(zip(xs,ys))) ## append coordinates to location's list of coordinates
    if loc['geometry']['type']=='Polygon': ## location is single part
        for coords in np.asarray(poly): ## iterate over coordinates
            coords=np.array(coords)
            xs=coords[:,0] ## longitudes
            ys=coords[:,1] ## latitudes

            location_points[location].append(np.vstack(zip(xs,ys))) ## append coordinates to location's list of coordinates

    complete_location=[]
    for part in location_points[location]: ## iterate over each component of a location
        complete_location.append(Polygon(part,True)) ## create a polygon for each component of a location

    polygons[location]=complete_location ## assign list of polygons to a location


print('polygons loaded:\n%s'%(polygons.keys()))




/Users/s1743989/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
/Users/s1743989/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


polygons loaded:
dict_keys(['kailahun', 'kenema', 'kono', 'bombali', 'kambia', 'koinadugu', 'portloko', 'tonkolili', 'bo', 'bonthe', 'moyamba', 'pujehun', 'westernarearural', 'westernareaurban'])


In [26]:
pop_centres = defaultdict(tuple)
aggregation = {}

with open("pop_centroids.txt") as f:
    next(f)
    for l in f:
        toks = l.strip("\n").split("\t")
        loc = toks[0].lower()
        
        if loc in locations:
        
            pop_centres[loc] = (float(toks[1]), float(toks[2]))
            
        elif loc == "westernrural":
            
            pop_centres["westernarearural"] = (float(toks[1]), float(toks[2]))
        
        elif loc == "westernurban":
            
            pop_centres["westernareaurban"] = (float(toks[1]), float(toks[2]))
        
        
print(pop_centres)

#Group A: Northern
#Group B: Eastern
#Group C: Southern
#Group D: Western

aggregation = {"bo": "group C", "bombali": "group A", "bonthe": "group C", "kailahun":"group B", "kambia": "group A",
              "kenema":"group B", "koinadugu": "group A", "kono": "group B", "moyamba": "group C", "portloko":"group A",
              "pujehun":"group C", "tonkolili":"group A", "westernarearural": "group D", "westernareaurban":"group D"}



defaultdict(<class 'tuple'>, {'bo': (-11.736966, 7.95880289), 'bombali': (-12.04188912, 8.89102396), 'bonthe': (-12.17397354, 7.60459324), 'kailahun': (-10.84492563, 7.99078795), 'kambia': (-12.91835852, 9.12335845), 'kenema': (-11.18956765, 7.85861409), 'koinadugu': (-11.55221977, 9.58822368), 'kono': (-10.97455967, 8.64463471), 'moyamba': (-12.07306903, 8.11096423), 'portloko': (-12.53675216, 8.68518875), 'pujehun': (-11.72051726, 7.35584565), 'tonkolili': (-12.21193614, 8.46548991), 'westernarearural': (-13.16852491, 8.44632216), 'westernareaurban': (-13.26196681, 8.47441962)})


In [31]:
border_sharing={locA:{locB:False for locB in polygons} for locA in polygons}

international_border=[]

for i,locA in enumerate(polygons): ## iterate over polygons
    for j,locB in enumerate(polygons): ## iterate over polygons again
        if i>j and aggregation[locA]!=aggregation[locB]: ## only do one pairwise comparison
            print(i,j,locA,locB)
            for partA in polygons[locA]: ## iterate over constituent polygons of location A
                setA=set(map(tuple,partA.get_verts())) ## convert coordinates of polygon into a set of tuples
                
                for partB in polygons[locB]:  ## iterate over constituent polygons of location B
                    setB=set(map(tuple,partB.get_verts())) ## convert coordinates of polygon into a set of tuples
                    temp=[] ## temporary storage
                    if setA.isdisjoint(setB)==False: ## if polygon A and polygon B share coordinates
                        
                        for point in map(tuple,partA.get_verts()): ## for each point in polygon A
                            if point in setA.intersection(setB) and point not in temp: ## if it is also present in polygon B and not seen yet - add to temporary list
                                temp.append(point)
                                
                    if len(temp)>0: ## if overlap between polygon A and B is not zero
                        international_border.append(temp) ## add to list of international bor

3 0 bombali kailahun
3 1 bombali kenema
3 2 bombali kono
4 0 kambia kailahun
4 1 kambia kenema
4 2 kambia kono
5 0 koinadugu kailahun
5 1 koinadugu kenema
5 2 koinadugu kono
6 0 portloko kailahun
6 1 portloko kenema
6 2 portloko kono
7 0 tonkolili kailahun
7 1 tonkolili kenema
7 2 tonkolili kono
8 0 bo kailahun
8 1 bo kenema
8 2 bo kono
8 3 bo bombali
8 4 bo kambia
8 5 bo koinadugu
8 6 bo portloko
8 7 bo tonkolili
9 0 bonthe kailahun
9 1 bonthe kenema
9 2 bonthe kono
9 3 bonthe bombali
9 4 bonthe kambia
9 5 bonthe koinadugu
9 6 bonthe portloko
9 7 bonthe tonkolili
10 0 moyamba kailahun
10 1 moyamba kenema
10 2 moyamba kono
10 3 moyamba bombali
10 4 moyamba kambia
10 5 moyamba koinadugu
10 6 moyamba portloko
10 7 moyamba tonkolili
11 0 pujehun kailahun
11 1 pujehun kenema
11 2 pujehun kono
11 3 pujehun bombali
11 4 pujehun kambia
11 5 pujehun koinadugu
11 6 pujehun portloko
11 7 pujehun tonkolili
12 0 westernarearural kailahun
12 1 westernarearural kenema
12 2 westernarearural kono
12 3

In [30]:
def desaturate(rgb,sat,lum):
    if isinstance(rgb,str):
        rgb=mpl.colors.hex2color(rgb) ## convert to rgb if string given
    elif np.array(rgb).shape[-1]==4: ## if there's four values
        rgb=rgb[:-1]
    elif isinstance(rgb,unicode):
        rgb=mpl.colors.hex2color(rgb) ## convert to rgb if string given

    assert 0.0<=sat<=1.0,'Desaturation outside allowed interval'
    assert np.array(rgb).shape[-1]==3,'Incorrect colour array shape'
    hsv=mpl.colors.rgb_to_hsv(rgb) ## convert rgb vector to hue, saturation, value
    hsv[1]=sat ## use provided saturation
    hsv[2]=lum ## use provided luminosity
    
    return mpl.colors.hsv_to_rgb(hsv) ## convert back to rgb

group_coordinates={}
normalised_coordinates={}
for high_order in set(aggregation.values()): ## iterate over high order group
    locs=[loc for loc in pop_centres.keys() if aggregation[loc]==high_order] ## get locations in group
    sorted_locations=sorted(locs,key=lambda loc:pop_centres[loc][0]) ## sort locations in group
    group_coordinates[high_order]=np.mean([pop_centres[loc] for loc in sorted_locations],axis=0)
    print(high_order,len(locs),[pop_centres[loc][0] for loc in locs])
    
    if len(locs)>2: ## for more than two locations colour coordinates based on their index along PCA1
        loc_pca = IncrementalPCA(n_components=2) ## create PCA object
        loc_pca.fit([pop_centres[loc] for loc in locs]) ## fit population centroid coordinates
        loc_transformed=list(map(tuple,loc_pca.transform([pop_centres[loc] for loc in locs]))) ## get PCA coordinates for population centroids
        sorted_transform=sorted(list(map(tuple,loc_transformed)),key=lambda x:x[0]) ## sort coordinates by x
        
        for l,loc in enumerate(locs):
            normalised_coordinates[loc]=sorted_transform.index(loc_transformed[l])/float(len(locs)-1)
    else:
        for l,loc in enumerate(locs):
            normalised_coordinates[loc]=(0.5+float(l))/float(len(locs))
        
sorted_group=sorted(list(set(aggregation.values())),key=lambda x:group_coordinates[x][0]) ## sort high-order groups according to longitude

ipca = IncrementalPCA(n_components=2) ## create PCA object
ipca.fit([group_coordinates[g] for g in sorted_group]) ## fit population centroid coordinates
transformed=ipca.transform([group_coordinates[g] for g in sorted_group]) ## get PCA coordinates for population centroids

XS=transformed[:,0] ## get PCA component 1

xnorm=mpl.colors.Normalize(0,len(XS)-1) ## create normalisation (index-based)

cmap=mpl.cm.viridis ## base colour map

group_colours={high_order:None for high_order in sorted_group}
for group,pos in zip(sorted_group,np.linspace(0.0,1.0,len(sorted_group))): ## iterate over transformed coordinates
    clist=[]
    clist.append([0.0,desaturate(cmap(pos),0.3,0.95)]) ## 0.0 in cmap is 30% saturated, 95% bright
    clist.append([0.5,desaturate(cmap(pos),0.6,0.8)]) ## 0.5 is 60% saturated, 80% bright
    clist.append([1.0,desaturate(cmap(pos),0.95,0.5)]) ## 1.0 is 95% saturated, 50% bright
    group_colours[group]=mpl.colors.LinearSegmentedColormap.from_list(group,clist) ## create colour map

fig,ax = plt.subplots(figsize=(15,14),facecolor='w') ## start figure

for loc in pop_centres.keys():
    x,y=pop_centres[loc]
    group=aggregation[loc]
    cmap=group_colours[group]
    c=cmap(normalised_coordinates[loc])
    
    ax.scatter(x,y,s=200,facecolor=c,edgecolor='k',zorder=20) ## plot population centre

    effects=[path_effects.Stroke(linewidth=4, foreground='white'),
                 path_effects.Stroke(linewidth=0.5, foreground='k')] ## black text, white outline

    ax.text(x,y,loc,va='bottom',ha='left',zorder=100,path_effects=effects) ## add its name
    
    
for i,loc in enumerate(polygons.keys()): ## iterate over locations
    ax.add_collection(PatchCollection(polygons[loc],facecolor='slategrey',edgecolor='w',lw=1,zorder=1)) ## add polygon

# for stretch in international_border: ## plot borders
#     xs,ys=zip(*stretch)
#     ax.plot(xs,ys,color='k',zorder=11,lw=2)
#     ax.plot(xs,ys,color='w',zorder=10,lw=5)

[ax.spines[loc].set_visible(False) for loc in ax.spines]
ax.tick_params(size=0)
ax.set_xticklabels([])
ax.set_yticklabels([])
    
ax.set_aspect(1)
plt.show()

fig,ax = plt.subplots(figsize=(len(sorted_group),4),facecolor='w') ## start figure
print(sorted_group)
for g,group in enumerate(sorted_group):
    for i in np.linspace(0.0,1.0,15):

        ax.scatter(g,i,s=700,color=group_colours[group](i),zorder=20) ## plot a column of points with colours of the colour map

ax.set_xticks(range(len(sorted_group)))
ax.set_xticklabels(sorted_group,rotation=90)

plt.show()

group A 5 [-12.04188912, -12.91835852, -11.55221977, -12.53675216, -12.21193614]
group B 3 [-10.84492563, -11.18956765, -10.97455967]
group D 2 [-13.16852491, -13.26196681]
group C 4 [-11.736966, -12.17397354, -12.07306903, -11.72051726]


findfont: Font family ['Helvetica Neue'] not found. Falling back to DejaVu Sans.


['group D', 'group A', 'group C', 'group B']


In [ ]:
tipSize=20
branchWidth=2

plt.figure(figsize=(20,10),facecolor='w') ## start figure
gs = gridspec.GridSpec(2, 1,height_ratios=[3,1],wspace=0.0,hspace=0.0) ## two row figure
ax1 = plt.subplot(gs[0]) ## ax1 is tree
ax2 = plt.subplot(gs[1],sharex=ax1) ## ax2 is a function for lineage presence circle radius

#This looks like it might all be for plotting the tree so could take it out
for k in objects: ## iterate over branches in tree
    loc=k.location ## get branch location
    group=aggregation[loc] ## get location's group
    cmap=group_colours[group] ## get group's colour map
    c=cmap(normalised_coordinates[loc]) ## get colour for location based on index of location along PCA1
    
    y=pop_centres[loc] ## y coordinate of branch
    yp=pop_centres[k.node_parent.location]
    
    x=k.absolute_time ## x coordinate of branch
    xp=k.node_parent.absolute_time ## x coordinate of parent branch
    
    if k.type=='Ind': ## if leaf
        ax1.scatter(x,y,s=tipSize,facecolor=c,edgecolor='none',zorder=101) ## plot colour tip circle
        ax1.scatter(x,y,s=tipSize*2,facecolor='k',edgecolor='none',zorder=100) ## black outline underneath every tip
        
    elif k.branchType=='Coal': ## if node
        yl=pop_centres[k.children[0].location] ## first child y coordinate
        
        ## For now, we only have direct children, so maybe I need to have that too
        yr=pop_centres[k.children[1].location]## last child y coordinate
        
        #Right so absolute time should actually be in terms of height? ATM for me it's in date format
        if xp==0.0:
            xp=x

        ax1.plot([x,x],[yl,yr],color=c,lw=branchWidth,zorder=98) ## plot vertical bar
    ax1.plot([xp,x],[y,y],color=c,lw=branchWidth,zorder=98) ## plot branch
    
ax1.tick_params(size=0,labelsize=0) ## no ticks or tick labels on all axes

circle_function={loc:None for loc in popCentres.keys()}

root_date=bt_tree.root.absoluteTime ## date of tree root
last_tip=root_date+bt_tree.treeHeight ## last tip date in tree
x_grid=np.linspace(root_date,last_tip,100) ## 100 equally spaced points between root and last tip
    
for loc in popCentres.keys(): ## for each location
    group=aggregation[loc] ## get group
    cmap=group_colours[group] ## get colour map
    c=cmap(normalised_coordinates[loc]) ## get location's colour
    
    ys=[len([k for k in bt_tree.Objects if k.traits[locTrait]==loc and k.parent.absoluteTime!=None and k.parent.absoluteTime<t<=k.absoluteTime]) for t in x_grid] ## calculate cumulative time spent in location
    ys=np.array(ys)/10.0 ## scale down
    spl=UnivariateSpline(x_grid,ys,s=0.1) ## use univariate spline...    
    
    circle_function[loc]=spl ## assign smoothed function to location
    ax2.plot(x_grid,circle_function[loc](x_grid),color=c) ## plot circle radii over time
    
plt.show()


plt.figure(figsize=(10,10),facecolor='w') ## start figure
gs = gridspec.GridSpec(1, 1,wspace=0.0,hspace=0.0)
ax3 = plt.subplot(gs[0]) ## ax3 is a map showing circle sizes at their maximum extent for each location

for loc in polygons.keys():
    ax3.add_collection(PatchCollection(polygons[loc],facecolor='w',edgecolor='grey')) ## plot polygons

for loc in popCentres.keys(): ## iterate over locations
    circle_radius=circle_function[loc] ## get circle radius function for location
    time_radii=[circle_radius(x) for x in x_grid if circle_radius(x)>0.0] ## calculate the radii for location over time, if they're not 0.0
    
    if len(time_radii)>0: ## if lineages are present
        radius_min,radius_max=min(time_radii),max(time_radii) ## get minimum and maximum radius values
    
        circle_min=plt.Circle(popCentres[loc],radius=radius_min,facecolor='steelblue',edgecolor='k',lw=1,zorder=101)
        circle_max=plt.Circle(popCentres[loc],radius=radius_max,facecolor='indianred',edgecolor='k',lw=1,zorder=100)
        ax3.add_patch(circle_min) ## plot circles at their minimum and maximum
        ax3.add_patch(circle_max)
ax3.plot() ## make sure polygons will be plotted
ax3.set_aspect(1) ## equal aspect

plt.show()

### Prepping tree

In [96]:
transm_dict = defaultdict(list)
nodes = []
times = []
child_dict = defaultdict(list)
location_dict = {}

with open(dropbox_path + results + log_file) as f:
    next(f)
    for l in f:
        toks = l.strip("\n").split(",")
        transm_dict[toks[0]] = [toks[1], int(toks[5]), int(toks[6])]
        
        nodes.append(toks[0])
        
        times.append(int(toks[7]))
        
        location_dict[toks[0]] = toks[4]
        
        child_dict[toks[1]].append(toks[0])

epidemic_len = max(times)

newick_tree, tree, R0 = cts.simulate_tree(transm_dict, child_dict, nodes, 0.16, epidemic_len)

objects = []

for i in tree.final_nodes:
    person = i.subtree.person.id
    i.location = location_dict[person]
    objects.append(i)

for i in tree.tips:
    i.location = location_dict[i.id]
    objects.append(i)
    


### Setting variables - need to do max cases here somewhere

In [138]:
def convert_date(x,start,end):
    """ Converts calendar dates between given formats """
    return dt.datetime.strftime(dt.datetime.strptime(x,start),end)

def decimal_date(input_date):
    start_year = dt.datetime.date(dt.datetime.strptime(str(input_date.year) +"-01-01", "%Y-%m-%d"))

    whole_diff = input_date-start_year

    day_diff = (whole_diff.days)/365

    decimal = input_date.year + day_diff

    return decimal



In [76]:
travelers=[x for x in objects if x.node_parent and x.location!=x.node_parent.location]

In [123]:
dates = []

start_date = dt.datetime.date(dt.datetime.strptime("2014-05-05", "%Y-%m-%d"))

for i in times:

    days = dt.timedelta(i)
    date = days + start_date
    
    dates.append(date)

In [125]:
epi_weeks = []

date = start_date

epi_weeks.append(date)

len_epidemic_weeks = round(max(times)/7)

print(len_epidemic_weeks)

for i in range(len_epidemic_weeks):
    change = dt.timedelta(7)
    
    new_date = date+change
    
    epi_weeks.append(new_date)
    
    date = new_date
    
    
print(epi_weeks)

12
[datetime.date(2014, 5, 5), datetime.date(2014, 5, 12), datetime.date(2014, 5, 19), datetime.date(2014, 5, 26), datetime.date(2014, 6, 2), datetime.date(2014, 6, 9), datetime.date(2014, 6, 16), datetime.date(2014, 6, 23), datetime.date(2014, 6, 30), datetime.date(2014, 7, 7), datetime.date(2014, 7, 14), datetime.date(2014, 7, 21), datetime.date(2014, 7, 28)]


In [140]:
for i in objects:
    day = tree.heights[i]*365
    i.absolute_time = decimal_date(start_date + dt.timedelta(day))
    
    

## Animation

In [121]:
t0 = time.time()
smooth = 2 #Number of gridpoints between each epiweek
dpi = 50
Bezier_smooth = 5
tracking_length = 21
depth = tracking_length/365.0

###
loc_trait = 'location.states'
#dates2 bit for only running animation for some dates for debugging
###



Nframes = len_epidemic_weeks*smooth 

animation_duration = 70
fps = int((Nframes)/animation_duration)

# height_normalisation needed - I think there's a bunch Bezier curve things I'll need to get out

In [147]:
def animate(frame):
    tr=(frame%smooth)/float(smooth) ## tr is a fraction of smoothing
    
    t=int(frame/smooth) ## t is index of time slice

    #### Primary plotting (map) ##I'LL NEED TO WORK THIS OUT
    ax1.lines=[line for line in ax1.lines if '_border' in line.get_label()] ## reset lines (except borders) and texts in the plot
    ax1.texts=[]
    ##
    
    if len(dates)-1>t: ## get epi week of next frame
        next_time=decimal_date(epi_weeks[t+1])
    else:
        next_time=decimal_date(epi_weeks[t])
    
    current_time=decimal_date(epi_weeks[t]) ## get epi week of current frame

    delta_time=next_time-current_time ## find interval step size

    ax1.text(0.05,0.1,'Epi week: %s\nDecimal time: %.3f'%(convert_date(epi_weeks[t],'%Y-%m-%d','%Y-%b-%d'),decimal_date(epi_weeks[t])+(delta_time*tr)),size=40,transform=ax1.transAxes) ## add text to indicate current time point
    
    #ax1.text(0.05,0.0,'@evogytis',size=28,ha='left',va='bottom',transform=ax1.transAxes)
    
    exists=[k for k in objects if k.node_parent and k.node_parent.absolute_time<=current_time+(delta_time*tr)<=k.absolute_time] ## identify lineages that exist at current timeslice

    lineage_locations=[c.location for c in exists]## identify locations where lineages are present
    lineage_counts = Counter(lineage_locations)
    presence = lineage_counts.keys()

    ###COME BACK TO THIS
    circle=[c.set_radius(0) for c in ax1.patches if '_circle' in c.get_label()] ## reset circle sizes
    ####
    
    for region in presence: ## iterate through every region where a lineage exists
        size=lineage_counts[region] ## count how many other lineages there are

        circle=[c for c in ax1.patches if c.get_label()=='%s_circle'%(region)][0] ## fetch circle at the location

        circle.set_radius(0.02+size*0.003) ## update its radius

    
    cur_slice=current_time+(delta_time*tr)

    for lineage in travelers: ## iterate through travelling lineages
        transition_time=(lineage.absolute_time+lineage.node_parent.absolute_time)/2.0 ## branch begins travelling mid-branch

        if cur_slice-depth<transition_time<cur_slice+depth: ## make sure transition is within period of animation
            frac=1-(transition_time-cur_slice)/float(depth) ## frac will go from 0.0 to 2.0

            origin=lineage.node_parent.location ## fetch locations
            destination=lineage.location

            point_a=pop_centres[origin] ## find coordinates of start and end locations
            begin_x,begin_y=point_a
            point_b=pop_centres[destination]
            end_x,end_y=point_b

            fc='k' ## colour line black unless it's travelling between countries

            #origin_country=location_to_country[origin] ## get countries for start and end points
            #destination_country=location_to_country[destination]
            
#             if origin_country!=destination_country: ## if lineage travelling internationally - colour it by origin country
#                 countryColour=colours[origin_country]
#                 fc=countryColour(0.6)

            distance=math.sqrt(math.pow(begin_x-end_x,2)+math.pow(begin_y-end_y,2)) ## calculate distance between locations

            ####
            normalized_height=height_normalization(cur_slice) ## normalize time of lineage
            ####
            
            adjust_d=-1+(1-normalized_height)+1/float(distance)**0.15+0.5 ## adjust Bezier line control point distance
            
            ####
            n=Bezier_control(pointA,pointB,adjust_d) ## find the coordinates of a point n that is at a distance adjust_d, perpendicular to the mid-point between points A and B
            ####
            
            bezier_start=frac-0.5 ## Bezier line begins at half a fraction along the path
            bezier_end=frac

            if bezier_start<0.0: ## if Bezier line begins outside the interval - make sure it stays within interval
                bezier_start=0.0
            if bezier_end>1.0:
                bezier_end=1.0

            bezier_line=Bezier([pointA,n,pointB],bezier_start,bezier_end,num=Bezier_smooth) ## get Bezier line points

            if bezier_start<1.0: ## only plot if line begins before destination
                for q in range(len(bezier_line)-1): ## iterate through Bezier line segments with fading alpha and reducing width
                    x1,y1=bezier_line[q]
                    x2,y2=bezier_line[q+1]

                    segL=(q+1)/float(len(bezier_line)) ## fraction along length of Bezier line
                    
                    ax1.plot([x1,x2],[y1,y2],lw=7*segL,alpha=1,color=fc,zorder=99,solid_capstyle='round') ## plot actual lineage

                    ax1.plot([x1,x2],[y1,y2],lw=10*segL,alpha=1,color='w',zorder=98,solid_capstyle='round') ## plot underlying white background to help lineages stand out

    
    #locations currently district list
    for i,loc in enumerate(locations):  ##plot new districts
        #some bits taken out here that I think I don't need
        
        if len(epi_weeks)-1>t:
            next_cases=cases_by_location[loc][epi_weeks[t+1]]
        else:
            next_cases=cases_by_location[loc][epi_weeks[t]]

        cur_cases=cases_by_location[loc][epi_weeks[t]]

        #country_max=1+float(maxByCountryTemporal[country]) ## get the maximum number of cases seen in the country at any point
        max_cases = 1 + max_cases
        
        interpolate=1+cur_cases+(next_cases-cur_cases)*tr ## interpolate between current and next cases (add one so that single cases show up after log normalization)     

        ####
        c=country_colour(np.log10(interpolate)/np.log10(country_max))
        #####
        
        polygons=[p for p in ax1.patches if p.get_label()=='%s_polygon'%(loc)]
        for polygon in polygons:
            polygon.set_facecolor(c) ## change the colour of locations based on cases
    
    frame+=1 ## next frame
    
    update=10 ## update progress bar every X frames
    
    
    
    
    
    

In [ ]:
### Not sure I'm going to use this, I think I'm more interested in just lineage mvmt rather than the tree too
    
    #### Secondary plotting (tree)
    Ls2=[x for x in ax2.lines if 'Colour' not in str(x.get_label())] ## fetch all the lines with labels in tree plot
    partials=[x for x in ax2.lines if 'partial' in str(x.get_label())]
    finished_lines=[x for x in ax2.lines if 'finished' in str(x.get_label())]
    finished_points=[x for x in ax2.collections if 'finished' in str(x.get_label())]
    
    finished_labels=[str(x.get_label()) for x in finished_lines]+[str(x.get_label()) for x in finished_points]
    partial_labels=[str(x.get_label()) for x in partials]
    
    if frame%update==0: ## progress bar
        clear_output()
        timeElapsed=(time.time() - t0)/60.0
        progress=int((frame*(50/float(Nframes))))
        percentage=frame/float(Nframes)*100
        rate=timeElapsed/float(frame)
        ETA=rate*(Nframes-frame)
        sys.stdout.write("[%-50s] %6.2f%%  frame: %5d %10s  time: %5.2f min  ETA: %5.2f min (%6.5f s/operation) %s %s %s" % ('='*progress,percentage,frame,dates2[t],timeElapsed,ETA,rate,len(partials),len(finished_lines),len(finished_points)))
        sys.stdout.flush()

        
    ####
    ## COMMENT this bit out if you don't want the tree to appear out of the time arrow
    ####
    for ap in ll.Objects:
        idx='%s'%(ap.index)
        xp=ap.parent.absoluteTime

        x=ap.absoluteTime
        y=ap.y

        location=ap.traits[locTrait]
        country=location_to_country[location]
        cmap=colours[country]
        c=cmap(normalized_coords[location])
        
        if xp<=cur_slice<x: ## branch is intersected
            if 'partial_%s'%(idx) in partial_labels: ## if branch was drawn before
                l=[w for w in partials if 'partial_%s'%(idx)==str(w.get_label())][-1]
                l.set_data([xp,cur_slice],[y,y])
            else: ## branch is intersected, but not drawn before
                ax2.plot([xp,cur_slice],[y,y],lw=branchWidth,color=c,zorder=99,label='partial_%s'%(ap.index))
                
        if x<=cur_slice: ## time arrow passed branch - add it to finished class
            if 'partial_%s'%(idx) in partial_labels:
                l=[w for w in partials if 'partial_%s'%(idx)==str(w.get_label())][-1]
                l.set_data([xp,x],[y,y])
                l.set_label('finished_%s'%(idx))
                finished_labels.append('finished_%s'%(idx))
                
            if 'finished_%s'%(idx) not in finished_labels:
                ax2.plot([xp,x],[y,y],lw=branchWidth,color=c,zorder=99,label='finished_%s'%(ap.index))
                
            if 'partial_%s'%(idx) in partial_labels or 'finished_%s'%(idx) not in finished_labels:
                if isinstance(ap,bt.leaf):
                    ax2.scatter(x,y,s=tipSize,facecolor=c,edgecolor='none',zorder=102,label='finished_%s'%(ap.index))
                    ax2.scatter(x,y,s=tipSize+30,facecolor='k',edgecolor='none',zorder=101,label='finished_%s'%(ap.index))
                elif isinstance(ap,bt.node):
                    yl=ap.children[0].y
                    yr=ap.children[-1].y
                    ax2.plot([x,x],[yl,yr],lw=branchWidth,color=c,zorder=99,label='finished_%s'%(ap.index))
    ####
    ## COMMENT this bit out if you don't want the tree to appear out of the time arrow
    ####
                
    for l in Ls2:
        if 'time' in l.get_label():
            l.set_data([cur_slice,cur_slice],[0,1]) ## adjust time arrow
            
        #### 
        ## UNCOMMENT this bit if you'd like lineages to be coloured over time
        ####
#         else:
#             ## fetch all line data
#             d_xs,d_ys=l.get_data()
            
#             ## extract x coordinate
#             start,end=d_xs
            
#             ## if time arrow passed end point of line - delete line
#             if end<cur_slice:
#                 ax2.lines.remove(l)
                
#             ## if time arrow passed start of line - adjust start of line
#             elif start<cur_slice:
#                 l.set_data([cur_slice,end],d_ys)
    
#     ## iterate over collections (scatter points) in tree plot
#     Ps2=[x for x in ax2.collections if 'Colour' not in str(x.get_label())]
    
#     for p in Ps2:
#         ## fetch coordinates
#         coords=p.get_offsets()
#         ## only alter points with 1 coordinate
#         if len(coords)==1:
#             ## remove black and white point if time arrow has passed
#             if coords[0][0]<=float(cur_slice):
#                 ax2.collections.remove(p)
        #### 
        ## UNCOMMENT this bit if you'd like lineages to be coloured over time
        ####
    
    ### Tertiary plotting (cases)
    Ls3=[x for x in ax3.lines if 'Colour' not in str(x.get_label())] ## fetch all the lines with labels in cases plot
    
    for l in Ls3:
        if 'time' in l.get_label():
            l.set_data([cur_slice,cur_slice],[0,1]) ## adjust time arrow
        else:
            d=l.get_xydata() ## fetch all line data
            
            for e in range(len(d)-1): ## iterate over points
                x_now=d[:,0][e] ## get coordinates of current and next positions
                x_nex=d[:,0][e+1]

                y_now=d[:,1][e]
                y_nex=d[:,1][e+1]
                
                if x_now<cur_slice: ## if beginning of line passed time arrow
                    d[:,0][e]=cur_slice # adjust coordinate so it's sitting on top of time arrow
                    d[:,1][e]=y_now+((y_nex-y_now)/(x_nex-x_now))*(cur_slice-x_now) 

In [ ]:


## This part will initialise the map, case numbers, and tree (in grey, if so set up)
plt.clf() 
plt.cla()
plt.figure(figsize=(32,18),facecolor='w') ## start figure

gs = gridspec.GridSpec(2, 2,width_ratios=[18,14],height_ratios=[14,4],hspace=0.05555,wspace=0.05882) ## define subplots

ax1 = plt.subplot(gs[0:, 0]) ## ax1 is map
ax2 = plt.subplot(gs[0, 1]) ## ax2 is tree
ax3 = plt.subplot(gs[1, 1]) ## ax3 is cases

for l,local_border in enumerate(global_border): ## plot the international borders
    ax1.plot(column(local_border,0),column(local_border,1),lw=5,color='w',zorder=96,label='%d_border_bg'%(l))
    ax1.plot(column(local_border,0),column(local_border,1),lw=2,color='k',zorder=97,label='%d_border'%(l))
    
for i,loc in enumerate(locations): ## iterate over locations, plot the initial setup
    country=location_to_country[loc]
    countryColour=colours[country]
    
    c=countryColour(0) ## zero cases colour

    if country in required_countries:
        N_lineages=plt.Circle(popCentres[loc],radius=0,label='%s_circle'%(loc),facecolor='indianred',edgecolor='k',lw=1,zorder=100) ## add circle that tracks the number of lineages at location with radius 0 to begin with
        ax1.add_patch(N_lineages)

        for part in location_points[loc]: ## plot every part of each location (islands, etc)
            poly=plt.Polygon(part,facecolor=c,edgecolor='grey',lw=1,label='%s_polygon'%(loc),closed=True,zorder=95)
            ax1.add_patch(poly)

ax1.spines['top'].set_visible(False) ## remove borders and axis labels
ax1.spines['right'].set_visible(False)
ax1.spines['left'].set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.tick_params(size=0)
ax1.set_xticklabels([])
ax1.set_yticklabels([])

ax1.set_ylim(ylimits) ## set plot limits
ax1.set_xlim(xlimits)

xlabels=['2013-%02d-01'%x for x in range(12,13)] ## setup time labels
xlabels+=['2014-%02d-01'%x for x in range(1,13)]
xlabels+=['2015-%02d-01'%x for x in range(1,13)]
xlabels+=['2016-%02d-01'%x for x in range(1,3)]

################
## Secondary plot begins - CASES
################
for c,country in enumerate(required_countries): ## iterate through countries
    greyColour=mpl.cm.Greys
    countryColour=colours[country]
    xs=[decimalDate(x) for x in dates] ## get time points based on epiweeks
    ys=[sum([cases_byLocation[loc][epiweek] for loc in locations if location_to_country[loc]==country]) for epiweek in dates] ## get cases in country at each epiweek
    
    grey_colour=greyColour((required_countries.index(country)+1)/float(len(required_countries)+2))
    
    ax3.plot(xs,ys,lw=3.3,color=grey_colour,zorder=2,label='BW') ## plot the same cases, one in full colour and one in grey on top to obscure colour
    ax3.plot(xs,ys,lw=3,color=countryColour(0.6),zorder=1,label='Colour')
    
ax3.axvline(decimalDate(dates[0]),color='k',lw=3,label='time',zorder=100) ## add time arrow to indicate current time

ax3.set_xticks([decimalDate(x)+1/24.0 for x in xlabels]) ## add ticks, tick labels and month markers
ax3.set_xticklabels([convertDate(x,'%Y-%m-%d','%b\n%Y') if x.split('-')[1]=='01' else convertDate(x,'%Y-%m-%d','%b') for x in xlabels])
[ax3.axvspan(decimalDate(xlabels[x]),decimalDate(xlabels[x])+1/12.,facecolor='k',edgecolor='none',alpha=0.04) for x in range(0,len(xlabels),2)]

ax3.xaxis.tick_bottom() ## make cases plot pretty
ax3.yaxis.tick_left()
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)

ax3.yaxis.set_major_locator(mpl.ticker.MultipleLocator(100))
ax3.set_xlim(decimalDate('2013-12-01'),decimalDate(dates2[-1]))
ax3.set_ylim(0,700)

ax3.tick_params(which='both',direction='out')
ax3.tick_params(axis='x',size=0,labelsize=18)
ax3.tick_params(axis='y',which='major',size=8,labelsize=30)
ax3.tick_params(axis='y',which='minor',size=5)
ax3.set_xticklabels([])
################
## Secondary plot ends - CASES
################


################
## Tertiary plot begins - TREE
################
tipSize=20
branchWidth=2

posteriorCutoff=0.0

####
## UNCOMMENT if you'd like the tree to be plotted in grey initially and get coloured over time
####
## iterate over objects in tree
# for k in ll.Objects:
#     location=k.traits[locTrait]
#     country=location_to_country[location]
#     cmap=colours[country]
#     c=cmap(normalized_coords[location])
    
#     countryColour=mpl.cm.Greys
#     grey_colour=countryColour((required_countries.index(country)+1)/float(len(required_countries)+2))
    
#     y=k.y
#     yp=k.parent.y
    
#     x=k.absoluteTime
#     xp=k.parent.absoluteTime
    
#     if isinstance(k,bt.leaf):
#         ## plot BW tree on top
#         ax2.scatter(x,y,s=tipSize,facecolor=grey_colour,edgecolor='none',zorder=102,label='LeafBW_%d'%(k.index))
#         ax2.scatter(x,y,s=tipSize+30,facecolor='k',edgecolor='k',zorder=100,label='Colour')
#         ax2.plot([xp,x],[y,y],color=grey_colour,lw=branchWidth,zorder=99,label='LeafBranchBW_%d'%(k.index))
        
#         ## plot colour tree underneath
#         ax2.scatter(x,y,s=tipSize,facecolor=c,edgecolor='none',zorder=101,label='LeafColour_%d'%(k.index))
#         ax2.plot([xp,x],[y,y],color=c,lw=branchWidth,zorder=98,label='LeafBranchColour_%d'%(k.index))
        
#     elif isinstance(k,bt.node):
#         yl=k.children[0].y
#         yr=k.children[-1].y
        
#         if xp==0.0:
#             xp=x

#         ls='-'
#         if k.traits['posterior']<posteriorCutoff:
#             ls='--'
            
#         ax2.plot([xp,x],[y,y],color=grey_colour,lw=branchWidth,ls=ls,zorder=99,label='NodeBranchBW_%d'%(k.index))
#         ax2.plot([x,x],[yl,yr],color=grey_colour,lw=branchWidth,ls=ls,zorder=99,label='NodeHbarBW_%d'%(k.index))
        
#         ax2.plot([xp,x],[y,y],color=c,lw=branchWidth,ls=ls,zorder=98,label='NodeBranchColour_%d'%(k.index))
#         ax2.plot([x,x],[yl,yr],color=c,lw=branchWidth,ls=ls,zorder=98,label='NodeHbarColour_%d'%(k.index))
####
## UNCOMMENT if you'd like the tree to be plotted in grey initially and get coloured over time
####

ax2.axvline(decimalDate(dates[0]),color='k',lw=3,label='time',zorder=200) ## add time arrow to indicate current time

ax2.set_xticks([decimalDate(x)+1/24.0 for x in xlabels]) ## add ticks, tick labels and month markers
ax2.set_xticklabels([convertDate(x,'%Y-%m-%d','%b\n%Y') if x.split('-')[1]=='01' else convertDate(x,'%Y-%m-%d','%b') for x in xlabels])
[ax2.axvspan(decimalDate(xlabels[x]),decimalDate(xlabels[x])+1/12.,facecolor='k',edgecolor='none',alpha=0.04) for x in range(0,len(xlabels),2)]

ax2.xaxis.tick_bottom() ## make tree plot pretty
ax2.yaxis.tick_left()
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax2.spines['bottom'].set_visible(False)

ax2.tick_params(axis='x',size=0)
ax2.tick_params(axis='y',size=0)
ax2.set_xticklabels([])
ax2.set_yticklabels([])

ax2.set_xlim(decimalDate('2013-12-01'),decimalDate(dates2[-1]))
ax2.set_ylim(-5,len(ll.Objects)/2.0+6)
################
## Tertiary plot ends - TREE
################

for i in range(0,Nframes): ## iterate through each frame
    animate(i) ## animate will modify the map, tree and cases
    plt.savefig(local_output+'EBOV_animation/ani_frame_%05d.png'%(i), format='png',bbox_inches='tight',dpi=dpi) ## save individual frames for stitching up using 3rd party software (e.g. FFMpeg)
    
print '\n\nDONE!'

## Expect a HUGE slow down around August 2014 (about 0.02 s/frame) due to lots of EBOV movement 
print '\nTime taken: %.2f minutes'%((time.time() - t0)/60.0)

fps=int((Nframes)/animation_duration)
print 'Recommended fps to get animation %d seconds long: %d'%(animation_duration,fps)
plt.show()